In [21]:
import glob
import dask.dataframe as dd
import pandas as pd

In [22]:
from dask.distributed import Client

client = Client()
client

C:\ProgramData\Anaconda3\lib\site-packages\distributed\bokeh\core.py:57: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn('\n' + msg)


Client Scheduler: tcp://127.0.0.1:52472 Dashboard: http://127.0.0.1:52475/status,Cluster Workers: 4 Cores: 16 Memory: 17.12 GB


In [23]:
csv_glob = 'COVID-19/csse_covid_19_data/csse_covid_19_daily_reports_us/*csv'
state_reports = dd.read_csv(csv_glob, dtype={'Active': 'float64'})

In [24]:
state_reports.head()

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3,Testing_Rate,Hospitalization_Rate
0,Alabama,US,2020-04-12 23:18:15,32.3182,-86.9023,3563,93,NaN,3470.0,1.0,75.988020,21583.0,437.0,2.610160,84000001,USA,460.300152,12.264945
1,Alaska,US,2020-04-12 23:18:15,61.3707,-152.4044,272,8,66.0,264.0,2.0,45.504049,8038.0,31.0,2.941176,84000002,USA,1344.711576,11.397059
2,Arizona,US,2020-04-12 23:18:15,33.7298,-111.4312,3542,115,NaN,3427.0,4.0,48.662422,42109.0,NaN,3.246753,84000004,USA,578.522286,NaN
3,Arkansas,US,2020-04-12 23:18:15,34.9697,-92.3731,1280,27,367.0,1253.0,5.0,49.439423,19722.0,130.0,2.109375,84000005,USA,761.753354,10.156250
4,California,US,2020-04-12 23:18:15,36.1162,-119.6816,22795,640,NaN,22155.0,6.0,58.137726,190328.0,5234.0,2.812020,84000006,USA,485.423868,22.961176


In [25]:
import bokeh

In [26]:
from bokeh.plotting import figure, output_notebook, show

In [27]:
state_reports.columns.tolist()

['Province_State',
 'Country_Region',
 'Last_Update',
 'Lat',
 'Long_',
 'Confirmed',
 'Deaths',
 'Recovered',
 'Active',
 'FIPS',
 'Incident_Rate',
 'People_Tested',
 'People_Hospitalized',
 'Mortality_Rate',
 'UID',
 'ISO3',
 'Testing_Rate',
 'Hospitalization_Rate']

In [28]:
state_reports = state_reports.compute()

In [29]:
output_notebook()

Loading BokehJS ...

In [30]:
#my_first_timeseries.py
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral3
output_file('simple_timeseries_plot.html')

#make sure MSNDATE is a datetime format
state_reports['Last_Update'] = dd.to_datetime(state_reports['Last_Update'], format='%Y-%m-%d')

grouped = state_reports.groupby(pd.Grouper(key='Last_Update', freq='D'))['Deaths', 'Recovered', 'Confirmed'].sum()
grouped = grouped/1000

source = ColumnDataSource(grouped)

p = figure(x_axis_type='datetime')

p.line(x='Last_Update', y='Deaths', line_width=2, source=source, legend='Deaths')
p.line(x='Last_Update', y='Recovered', line_width=2, source=source, color=Spectral3[1], legend='Recovered')
p.line(x='Last_Update', y='Confirmed', line_width=2, source=source, color=Spectral3[2], legend='Confirmed')

p.yaxis.axis_label = 'test'

show(p)